# LLA

## Notes for this notebook

* Running `LLA` on the `tony` sequencing run.
* The `georg` sequencing run will be processed in a separate notebook, then the data will be merged.

## General notes

Ley Lab Amplicon pipeline (LLA)

This notebook describes the standard operating procedure for generating sequence variants (SVs) from raw amplicon sequence data.

***

**This notebook is set up to process the following data:**

* demultiplexed MiSeq paired-end sequences (raw fastq files)

If your data differs from this, you may need to heavily modify this notebook in order to process your amplicon data.

## Preparing your data

Note that sequence data generated at the MPI Genome Center is already demultiplexed. The sequence data is stored at the [local SRA](http://sra.eb.local/sra/index.html). You should **symlink** all of your sequence data (the `fastq.gz` files) into 1 directory.

An example for creating a symlink: 

`ln -s /path/to/MY_FILE.fastq.gz /new/path/to/MY_FILE.fastq.gz`

**Note:** the directory should ONLY contain the read files that you want to import

## Setting parameters

### Location of read files

Your read files should all be in the same directory. QIIME2 will automatically find all read files in the directory. If you need to combine read files from multiple locations, it is best in most cases to create symlinks instead of copying the read files (this creates unnesessary redundant file copies). 

In [7]:
import os
read_file_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/'
tony_read_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/raw_run_data/tony/qiime2_import/'
tony_read1_file = os.path.join(tony_read_dir, 'forward.fastq.gz')
tony_read2_file = os.path.join(tony_read_dir, 'reverse.fastq.gz')
tony_index_file = os.path.join(tony_read_dir, 'barcodes.fastq.gz')

### Working directory

Where do you want to generate all output files?

In [8]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/'

### Data type

In the particular case of the mock community reads, the data type that fits our needs is `SampleData[PairedEndSequencesWithQuality]`. You may also have paired-end sequencing data with barcoded samples. In that case, the data type to chose is `EMPPairedEndSequences`.

In [9]:
data_type = 'SampleData[PairedEndSequencesWithQuality]'

### Classifier

Database used for taxonomy classification. The default here is `Silva 119 99% OTUs from 515F/806R region of sequences`. See the [QIIME2 docs](https://docs.qiime2.org/2017.10/data-resources/#taxonomy-classifiers-for-use-with-q2-feature-classifier) for info on other databases.  

In [10]:
classifier = '/ebio/abt3_projects/databases/leylab16s/classifiers/silva-119-99-515-806-nb-classifier.qza'

### Compute cluster parameters

These will be the default parameters used for all compute cluster jobs (SGE jobs)

In [11]:
# MAKE SURE to CHANGE this to your email!
email='nyoungblut@tuebingen.mpg.de'

# only used for multi-threaded steps
threads=20   

# Note: memory is per-thread (eg., 8 threads x 8G memory = 64G total memory)
memory='3G'

# Max job time 
job_time='24:0:0'

# Output for SGE job errors/warnings/info
SGE_out='~/SGE/LLA/'

# Path to conda installation
conda_env_path='/ebio/abt3_projects/software/miniconda3' 

# Particular conda environment to use
conda_env='qiime2'

***
**You shouldn't need to modify anything below this line**
*** 

# Initialize

Importing python packages, defining functions, etc.

In [12]:
import os
import time

In [13]:
# making directories (if they don't exist)

## working directory
work_dir = os.path.abspath(os.path.expanduser(work_dir))
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

## SGE output    
SGE_out = os.path.abspath(os.path.expanduser(SGE_out))
if not os.path.isdir(os.path.split(SGE_out)[0]):
    os.makedirs(SGE_out)

In [14]:
# changing directory
%cd $work_dir

/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony


## Functions

Defining python functions

In [15]:
def write_file(s, file_name):
    """Writing out (multi-line) string to file
    """
    F = os.path.abspath(os.path.expanduser(file_name))
    with open(F, 'w') as outF:
        outF.write(s)
    print('File written: {}'.format(F))

In [16]:
def sge_job(job_name='LLA', threads=threads, email=email, 
            memory=memory, job_time=job_time, SGE_out=SGE_out,
            conda_env_path=conda_env_path, 
            conda_env=conda_env):
    """Creating an SGE job script template 
    """
    job = """#!/bin/bash
#$ -N {job_name}
#$ -pe parallel {threads}
#$ -l h_vmem={memory}
#$ -l h_rt={job_time}
#$ -o {SGE_out}
#$ -j y
#$ -cwd
#$ -m ea
#$ -M {email}

CONDA_INSTALLATION="{conda_env_path}"
QIIME2_ENV="{conda_env}"

export PATH="$CONDA_INSTALLATION/bin":$PATH
export PATH="$CONDA_INSTALLATION/envs/$QIIME2_ENV/bin":$PATH
export LC_ALL=C.UTF-8
export LANG=C.UTF-8

""".format(job_name=job_name, threads=threads, email=email,
           memory=memory, job_time=job_time, SGE_out=SGE_out,
           conda_env_path=conda_env_path, conda_env=conda_env)
    return job

In [17]:
def qsub_wait(file_name):
    """Submit SGE job via `qsub` then wait for the job to finish (success or abort)
    """
    # submit job
    ret = !! qsub $file_name
    # get job ID
    job_ID = ret[0].split(' ')[2]
    print('SGE Job ID: {}'.format(job_ID))
    
    # query job
    while 1:
        ret = !! qstat
        IDs = [x.lstrip().split(' ')[0] for x in ret]
        if job_ID in IDs:
            time.sleep(2)
            continue
        else:
            print('SGE job finished: {}'.format(job_ID))
            break  

## Trimmomatic

Before we can run `dada2` to call sequence variants, we need to first determine which positions in the reads to trim off due to low quality score (on average across reads). `dada2` requires this information in order to call sequence variants correctly. 

One could look at the sequence quality score distribution plots generated by `qiime demux summarize`; however, this method is subjective and will differ among researchers. To keep the data processing pipeline consistent, we use `trimmomatic` to determine the position in the reads where average quality scores drop below a specific cutoff.

To determine the cutoff positions with Trimmomatic, we will use the following script. It provides a summary of the lengths of the reads after being trimmed by using a sliding window approach. The script will generate read length cutoffs for both the forward and reverse reads.

### Creating trimmomatic pipeline script

In [18]:
%%writefile subsample_trim.sh
#!/bin/bash

# user input
if [ "$#" -lt 5 ]; then
    echo "Usage: trim.sh read_file_dir sample_size window_size qlty_threshold threads"
    echo "Description: randomly subsample from all reads, then run trimmotic to get a quality score cutoff"
    exit
fi

# ===== INPUT PARAMETERS ===== 

FASTQ_DIR=$1
SAMPLE_SIZE=$2
WINDOW_SIZE=$3
QLTY_TRESHOLD=$4
THREADS=$5

# === MAIN ===

declare -a READS_FILES=("forward" "reverse")

for READS_TO_USE in "${READS_FILES[@]}"
do
    if [ $READS_TO_USE == 'forward' ]
    then
        read="forward.fastq.gz"
    elif [ $READS_TO_USE == 'reverse' ]
    then
        read="reverse.fastq.gz"
    fi
    
    # Random subsampling
    [ -e merged_${READS_TO_USE}.fastq ] && rm merged_${READS_TO_USE}.fastq
    for filename in "$FASTQ_DIR/$read"; do
        seqtk sample -s11 ${filename} ${SAMPLE_SIZE} >> merged_${READS_TO_USE}.fastq
    done

    # Trimming of the reads
    [ -e trimmed_${READS_TO_USE}.fastq ] && rm trimmed_${READS_TO_USE}.fastq
    [ -e trimmed_${READS_TO_USE}.log ] && rm trimmed_${READS_TO_USE}.log
    trimmomatic SE \
        merged_${READS_TO_USE}.fastq \
        trimmed_${READS_TO_USE}.fastq \
        SLIDINGWINDOW:${WINDOW_SIZE}:${QLTY_TRESHOLD} -trimlog trimmed_${READS_TO_USE}.log -threads $THREADS
    
    # Recover the surviving sequence length of the trimmed reads
    [ -e trim_results_${READS_TO_USE}.txt ] && rm trim_results_${READS_TO_USE}.txt
    awk -F ' ' '{print $(NF-3)}' trimmed_${READS_TO_USE}.log >> trim_results_${READS_TO_USE}.txt

    [ -e trim_median_${READS_TO_USE}.txt ]  && rm trim_median_${READS_TO_USE}.txt
    sort -n trim_results_${READS_TO_USE}.txt | \
        awk '{ count[NR]=$1; } END { \
        if (NR % 2) { print count[(NR + 1) / 2]; } \
        else { print (count[(NR / 2)] + count[(NR / 2) + 1]) / 2.0; } }' > trim_median_${READS_TO_USE}.txt
done

Overwriting subsample_trim.sh


### Submitting the SGE job

In [19]:
cmd = sge_job(job_name='trim') 
cmd = cmd + """
bash subsample_trim.sh {read_file_dir} {sample_size} {window_size} {qlty_threshold} {threads}
"""
cmd = cmd.format(read_file_dir=tony_read_dir, 
                 sample_size=10000, 
                 window_size=20, 
                 qlty_threshold=25, 
                 threads=threads)

write_file(cmd, 'trim.sh')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/trim.sh


In [20]:
# view the job script
!cat trim.sh

#!/bin/bash
#$ -N trim
#$ -pe parallel 20
#$ -l h_vmem=3G
#$ -l h_rt=24:0:0
#$ -o /ebio/abt3/nyoungblut/SGE/LLA
#$ -j y
#$ -cwd
#$ -m ea
#$ -M nyoungblut@tuebingen.mpg.de

CONDA_INSTALLATION="/ebio/abt3_projects/software/miniconda3"
QIIME2_ENV="qiime2"

export PATH="$CONDA_INSTALLATION/bin":$PATH
export PATH="$CONDA_INSTALLATION/envs/$QIIME2_ENV/bin":$PATH
export LC_ALL=C.UTF-8
export LANG=C.UTF-8


bash subsample_trim.sh /ebio/abt3_projects/Georg_animal_feces/data/16S/raw_run_data/tony/qiime2_import/ 10000 20 25 20


In [21]:
# submit to the cluster and wait until job completion/abort
qsub_wait('trim.sh')

SGE Job ID: 718435
SGE job finished: 718435


# Step 3: Sequence Variant selection with DADA2

**No more OTU picking! No more 97% OTUs!**

[DADA2](https://www.ncbi.nlm.nih.gov/pubmed/27214047) is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data, generating amplicon sequence variants (ASV or SV). The `dada2 denoised-paired` method requires two parameters that are used in quality filtering: `--p-trunc-len-f n` and `--p-trunc-len-r m` which truncate both the forward and reverse reads at the indicated positions `n` and `m`; this allows the user to remove low quality regions of the sequences. In our case, `n = m`, although you may select different values for `n` and `m`.

To determine the values to pass for these two parameters, you should review the *Interactive Quality Plot* tab in the [QIIME2 Viewer](https://view.qiime2.org/) for the summary file (`demux.qzv`) obtained before. You can also chose to use the standard value that the Leylab 16S pipeline determines.

**Note:** `dada2` may require *many hours* to finish, depending on the number of samples, number of threads, etc.

In [22]:
# trimmomatic-defined read quality cutoff position
fwd_read_len = !! cat trim_median_forward.txt
fwd_read_len[0]

'249'

In [23]:
# trimmomatic-defined read quality cutoff position
rev_read_len = !! cat trim_median_reverse.txt
rev_read_len[0]

'194'

In [24]:
cmd = sge_job(job_name='dada2') 
cmd = cmd + """
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs tony_demux.qza \
    --p-trim-left-f 19 \
    --p-trim-left-r 20 \
    --p-trunc-len-f {fwd_read_len} \
    --p-trunc-len-r {rev_read_len} \
    --p-n-threads {threads} \
    --o-representative-sequences rep-seqs.qza \
    --o-table table.qza
"""
cmd = cmd.format(fwd_read_len=fwd_read_len[0], 
                 rev_read_len=rev_read_len[0],
                 threads=threads)

write_file(cmd, 'dada2.sh')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/dada2.sh


In [25]:
# view job script
!cat dada2.sh

#!/bin/bash
#$ -N dada2
#$ -pe parallel 20
#$ -l h_vmem=3G
#$ -l h_rt=24:0:0
#$ -o /ebio/abt3/nyoungblut/SGE/LLA
#$ -j y
#$ -cwd
#$ -m ea
#$ -M nyoungblut@tuebingen.mpg.de

CONDA_INSTALLATION="/ebio/abt3_projects/software/miniconda3"
QIIME2_ENV="qiime2"

export PATH="$CONDA_INSTALLATION/bin":$PATH
export PATH="$CONDA_INSTALLATION/envs/$QIIME2_ENV/bin":$PATH
export LC_ALL=C.UTF-8
export LANG=C.UTF-8


qiime dada2 denoise-paired     --i-demultiplexed-seqs tony_demux.qza     --p-trim-left-f 19     --p-trim-left-r 20     --p-trunc-len-f 249     --p-trunc-len-r 194     --p-n-threads 20     --o-representative-sequences rep-seqs.qza     --o-table table.qza


In [26]:
# submit to the cluster and wait until job completion/abort
qsub_wait('dada2.sh')

SGE Job ID: 718566
SGE job finished: 718566


**Note:** if the step above fails, you may need to increase the memory designation for the compute cluster job

# Step 4. Visualize SV table

At this stage, you will hopefully have artifacts (remember: this are files, not sources of experimental bias) containing the feature table and corresponding feature sequences. You can summarize them as follows

## Feature table

This step will generate visual and tabular summaries of the SV table

In [27]:
cmd = sge_job(job_name='feature_table', threads=1) 
cmd = cmd + """
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv
"""

write_file(cmd, 'feature_table.sh')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/feature_table.sh


In [28]:
# visualize job script
!cat feature_table.sh

#!/bin/bash
#$ -N feature_table
#$ -pe parallel 1
#$ -l h_vmem=3G
#$ -l h_rt=24:0:0
#$ -o /ebio/abt3/nyoungblut/SGE/LLA
#$ -j y
#$ -cwd
#$ -m ea
#$ -M nyoungblut@tuebingen.mpg.de

CONDA_INSTALLATION="/ebio/abt3_projects/software/miniconda3"
QIIME2_ENV="qiime2"

export PATH="$CONDA_INSTALLATION/bin":$PATH
export PATH="$CONDA_INSTALLATION/envs/$QIIME2_ENV/bin":$PATH
export LC_ALL=C.UTF-8
export LANG=C.UTF-8


qiime feature-table summarize   --i-table table.qza   --o-visualization table.qzv


In [29]:
# submit to the cluster and wait until job completion/abort
qsub_wait('feature_table.sh')

SGE Job ID: 731505
SGE job finished: 731505


## Rep seqs

This step will generate tabular view of feature identifier to sequence mapping, including links to BLAST each sequence against the NCBI nt database.

In [30]:
cmd = sge_job(job_name='req_seqs', threads=1) 
cmd = cmd + """
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv
"""

write_file(cmd, 'rep_seqs.sh')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/rep_seqs.sh


In [31]:
# view the job script before submission
!cat rep_seqs.sh

#!/bin/bash
#$ -N req_seqs
#$ -pe parallel 1
#$ -l h_vmem=3G
#$ -l h_rt=24:0:0
#$ -o /ebio/abt3/nyoungblut/SGE/LLA
#$ -j y
#$ -cwd
#$ -m ea
#$ -M nyoungblut@tuebingen.mpg.de

CONDA_INSTALLATION="/ebio/abt3_projects/software/miniconda3"
QIIME2_ENV="qiime2"

export PATH="$CONDA_INSTALLATION/bin":$PATH
export PATH="$CONDA_INSTALLATION/envs/$QIIME2_ENV/bin":$PATH
export LC_ALL=C.UTF-8
export LANG=C.UTF-8


qiime feature-table tabulate-seqs   --i-data rep-seqs.qza   --o-visualization rep-seqs.qzv


In [32]:
# submit to the cluster and wait until job completion/abort
qsub_wait('rep_seqs.sh')

SGE Job ID: 731506
SGE job finished: 731506


The resulting table is a summary of the SV selection results